# DMRG

In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pyscf
from pyscf import gto,scf,fci,lo,mcscf
from pyscf import ao2mo
from pyscf.tools import molden

/Users/zhendongli/.local/lib/python3.9/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


# Preparation of integrals

In [122]:
molname = 'h2o'
mo = 'cmo'
basis = 'sto-3g' #cc-pvdz'
natoms = 16
re = 0.98
fac = 5.0
r = fac*re #5.0
theta = 104.5*np.pi/180 # degree to rad

mol = gto.Mole()
mol.verbose = 0

if molname == 'hchain':
    mol.atom = [['H',(0.0,0.0,i*r)] for i in range(natoms)]

elif molname == 'h2o':
    mol.atom = [['O',(0,0,0)],
                ['H',(0,r,0)],
                ['H',(0,r*np.cos(theta),r*np.sin(theta))]]
    
elif molname == 'hplane':
    mol.atom = [['H',(0.,0.,0.)],
                ['H',(0.,0.,r)],
                ['H',(0.,0.,2*r)],
                ['H',(0.,r,0)],
                ['H',(0.,r,r)],
                ['H',(0.,r,2*r)]]
    
elif molname == 'n2':
    mol.atom = [['N',(0.,0.,0.)],
                ['N',(0.,0.,r)]]

elif molname == 'hf':
    mol.atom = [['H',(0.,0.,0.)],
                ['F',(0.,0.,r)]]

elif molname == 'c2':
    mol.atom = [['C',(0.,0.,0.)],
                ['C',(0.,0.,r)]]

elif molname == 'h18':
    atoms = []
    for k in range(2):
        for j in range(3):
            for i in range(3):
                x = i * r
                y = j * r
                z = k * r
                atoms += [['H', (x,y,z)]]
    mol.atom = atoms
    mol.unit = 'bohr'

elif molname == 'h36':
    atoms = []
    for k in range(4):
        for j in range(3):
            for i in range(3):
                x = i * r
                y = j * r
                z = k * r
                atoms += [['H', (x,y,z)]]
    mol.atom = atoms
    mol.unit = 'bohr'

elif molname == 'h50':
    mol.atom = [['H',(0.0,0.0,i*r)] for i in range(50)]
    mol.unit = 'bohr'
    
mol.basis = basis
mol.symmetry = True 
mol.charge = 0
mol.spin = 0
mol.build()

mf = scf.RHF(mol)
mf.init_guess = 'atom'
mf.level_shift = 0.0
mf.max_cycle = 100
mf.conv_tol=1.e-14
mf.scf()

-72.64445852980812

In [123]:
#
# FCI
#
if mo == 'cmo':
    mo_coeff = mf.mo_coeff
elif mo == 'oao':
    mo_coeff = lo.orth_ao(mol, method="meta-lowdin")
elif mo == 'lowdin':
    mo_coeff = lo.orth_ao(mol, method="lowdin")

molden.from_mo(mol, 'mo.molden', mo_coeff)

norb = mo_coeff.shape[1]
h1 = mo_coeff.T.dot(mf.get_hcore()).dot(mo_coeff)
eri = ao2mo.kernel(mol, mo_coeff)

# cisolver = fci.direct_spin1.FCI(mol)
# cisolver = pyscf.fci.addons.fix_spin(cisolver,shift=0.1,ss=0)
# cisolver.nroots = 3 #10
# norb = mol.nao
# nelec = [mol.nelectron//2,mol.nelectron//2]
# elst, ci = cisolver.kernel(h1, eri, norb, nelec, ecore=mol.energy_nuc(), max_cycle=1000, tol=1.e-16)
# for e in elst:
#     print(e)

In [124]:
from ordering_and_graph import fielder

eri = ao2mo.general(mol,(mo_coeff,mo_coeff,mo_coeff,mo_coeff),compact=0).reshape(norb,norb,norb,norb)
kij = np.einsum('ijji->ij',eri)
forder = fielder.orbitalOrdering(kij,mode='kmat',debug=False)
for i in forder:
    print(i)
print('no. of orbitals=',len(forder))

2
1
3
4
5
6
0
no. of orbitals= 7


# FOCUS calculations

In [266]:
import focus_class

os.chdir('/Users/zhendongli/Desktop/FOCUS/pyfocus/')
common = focus_class.Common()
common.parrentdir = os.getcwd()
common.workdir = 'tmp'
common.dtype = 0
common.nelec = mol.nelectron
common.twoms = 0
common.integral_file = './h2o_'+str(fac)+'.info'
common.scratch = 'scratch'
common.build()

parrentdir = /Users/zhendongli/Desktop/FOCUS/pyfocus
workdir = /Users/zhendongli/Desktop/FOCUS/pyfocus/tmp


0

In [267]:
import ipyscf_real

iface = ipyscf_real.iface()
iface.mol = mol
iface.mf = mf
iface.nfrozen = 0
info = iface.get_integral(mo_coeff[:,forder])
iface.dump(info,fname=common.workdir+'/h2o_'+str(fac)+'.info')


[iface.get_integral]
finished

[iface.dump] fname= /Users/zhendongli/Desktop/FOCUS/pyfocus/tmp/h2o_5.0.info
finished


0

In [268]:
sci = focus_class.SCI(common)
sci.dets = [range(mol.nelectron)]
sci.nroots = 2
sci.eps0 = 1.e-4
sci.maxiter = 1
sci.schedule = [(0,1.e-3)]
sci.kernel(fname='sci.dat',output='sci.out',iprt=0)


SCI calculations: sci.x sci.dat > sci.out


[-72.7167904401, -72.715191982863]

In [269]:
ctns = focus_class.CTNS(common)
ctns.qkind = 'rNSz'
ctns.nroots = 1
ctns.maxdets = 100
ctns.thresh_proj = 1.e-14
ctns.topology_file = 'topo'
ctns.topo = range(mol.nao)
ctns.schedule = [(0,2,64,1.e-4,1.e-8)]
ctns.maxsweep = 4
ctns.tasks = ['task_dmrg']
ctns.alg_hvec = 4
ctns.alg_renorm = 4
ctns.kernel(fname='ctns.dat',output='ctns.out',iprt=0)


CTNS calculations: ctns.x ctns.dat > ctns.out


[[-74.6424250456], [-74.6424250456], [-74.6424250456], [-74.6424250456]]

In [270]:
ctns.savebin(isweep=1,iprt=0)


SAVEBIN: prop.x savebin.dat > savebin.out


0

# Analysis of the final MPS

In [191]:
import ctns_loader

ctns = ctns_loader.ctns_info()

ctns.load('./rcanon_isweep1.bin')

sites = ctns.toMPSdense()

In [192]:
import mps_simple

iroot = 0
shapes = mps_simple.shapes(sites)
print('shapes=\n',shapes)
ova = mps_simple.overlap(sites,sites)
print('ova=\n',ova)
spMPS = mps_simple.singleSiteEntropy(sites,iroot)
print('spMPS=\n',np.sum(spMPS),spMPS)
spqMPS = mps_simple.twoSiteEntropy(sites,iroot)
print('spqMPS=\n',spqMPS)
IpqMPS = mps_simple.mutualInformation(spqMPS,spMPS)
print('IpqMPS=\n',IpqMPS)

shapes=
 [(1, 4, 4), (4, 4, 14), (14, 4, 16), (16, 4, 64), (64, 4, 16), (16, 4, 4), (4, 4, 1)]
ova=
 [[1.]]
spMPS=
 2.889178422089349 [6.93147413e-01 6.93147795e-01 2.13741588e-04 6.54022000e-01
 2.41399612e-01 6.07247853e-01 7.50591944e-09]
spqMPS=
 [[0.                nan 0.69336115 1.34716903 0.93454698 1.30039527
  0.69314742]
 [       nan 0.         0.69336109 1.34716972 0.93454695 1.30039565
  0.6931478 ]
 [0.69336115 0.69336109 0.                nan        nan        nan
         nan]
 [1.34716903 1.34716972        nan 0.                nan        nan
         nan]
 [0.93454698 0.93454695        nan        nan 0.                nan
         nan]
 [1.30039527 1.30039565        nan        nan        nan 0.
         nan]
 [0.69314742 0.6931478         nan        nan        nan        nan
  0.        ]]
IpqMPS=
 [[0.00000000e+00            nan 5.27001220e-10 3.78050041e-07
  4.29587820e-08 1.04466547e-09 1.23789867e-13]
 [           nan 0.00000000e+00 4.48490440e-07 7.28538510e-08
 

/Users/zhendongli/Desktop/FOCUS/pyfocus/mps_simple.py:26: RuntimeWarning: divide by zero encountered in log
  s += -p*np.log(p)
/Users/zhendongli/Desktop/FOCUS/pyfocus/mps_simple.py:26: RuntimeWarning: invalid value encountered in scalar multiply
  s += -p*np.log(p)


## difference compared against FCI

In [4]:
np.linalg.norm(spMPS - sp)

NameError: name 'sp' is not defined

In [5]:
np.linalg.norm(spqMPS - spq)

NameError: name 'spq' is not defined

In [30]:
np.linalg.norm(IpqMPS - Ipq)

NameError: name 'Ipq' is not defined

In [31]:
Ipq

NameError: name 'Ipq' is not defined

In [32]:
spqMPS

array([[0.        , 0.32061799, 0.43411703, 0.44137604, 0.31160085,
        0.17173139],
       [0.32061799, 0.        , 0.4620307 , 0.45725296, 0.23796873,
        0.3103511 ],
       [0.43411703, 0.4620307 , 0.        , 0.29184435, 0.45445576,
        0.42529977],
       [0.44137604, 0.45725296, 0.29184435, 0.        , 0.46607903,
        0.43637053],
       [0.31160085, 0.23796873, 0.45445576, 0.46607903, 0.        ,
        0.30414783],
       [0.17173139, 0.3103511 , 0.42529977, 0.43637053, 0.30414783,
        0.        ]])

In [33]:
spqMPS - spq

NameError: name 'spq' is not defined

# Canonicalization

In [193]:
mps_simple.checkRCF(sites)

check i= 6  site.shape= (4, 4, 1)  |S-I|= 0.0
check i= 5  site.shape= (16, 4, 4)  |S-I|= 0.0
check i= 4  site.shape= (64, 4, 16)  |S-I|= 0.0
check i= 3  site.shape= (16, 4, 64)  |S-I|= 1.8895263174248027e-15
check i= 2  site.shape= (14, 4, 16)  |S-I|= 9.28391608533688e-16
check i= 1  site.shape= (4, 4, 14)  |S-I|= 8.15843973306311e-16
check i= 0  site.shape= (1, 4, 4)  |S-I|= 0.0
MPS is in RCF


0

In [7]:
mps_simple.checkLCF(sites)

check i= 0  site.shape= (2, 4, 8)  |S-I|= 2.426946616543995
check i= 1  site.shape= (8, 4, 32)  |S-I|= 4.686096540779159
check i= 2  site.shape= (32, 4, 64)  |S-I|= 5.9215501870931595
check i= 3  site.shape= (64, 4, 16)  |S-I|= 12.0
check i= 4  site.shape= (16, 4, 4)  |S-I|= 6.0
check i= 5  site.shape= (4, 4, 1)  |S-I|= 3.0
MPS is not in LCF: diff= 34.034593344416315


1

In [10]:
sites_lcf0 = mps_simple.getLCFStateFromRCF(sites, 0)
mps_simple.checkLCF(sites_lcf0)

check i= 0  site.shape= (1, 4, 4)  |S-I|= 0.0
check i= 1  site.shape= (4, 4, 16)  |S-I|= 3.7903524883928246e-15
check i= 2  site.shape= (16, 4, 64)  |S-I|= 1.3728164257946061e-14
check i= 3  site.shape= (64, 4, 32)  |S-I|= 7.300643707857429e-15
check i= 4  site.shape= (32, 4, 8)  |S-I|= 1.494338323643004e-15
check i= 5  site.shape= (8, 4, 1)  |S-I|= 3.1086244689504383e-15
MPS is in LCF


0

In [11]:
sites_lcf1 = mps_simple.getLCFStateFromRCF(sites, 1)
mps_simple.checkLCF(sites_lcf1)

check i= 0  site.shape= (1, 4, 4)  |S-I|= 0.0
check i= 1  site.shape= (4, 4, 16)  |S-I|= 3.7903524883928246e-15
check i= 2  site.shape= (16, 4, 64)  |S-I|= 1.3728164257946061e-14
check i= 3  site.shape= (64, 4, 32)  |S-I|= 7.300643707857429e-15
check i= 4  site.shape= (32, 4, 8)  |S-I|= 1.494338323643004e-15
check i= 5  site.shape= (8, 4, 1)  |S-I|= 2.220446049250313e-16
MPS is in LCF


0

In [12]:
mps_simple.overlap(sites_lcf0, sites_lcf1)

array([[-1.56087392e-15]])

In [14]:
sites_rcf0 = mps_simple.getRCFStateFromRCF(sites, 0)
mps_simple.overlap(sites_lcf0, sites_rcf0)

array([[1.]])

# Dump for FOCUS

In [194]:
mps_simple.shapes(sites)

[(1, 4, 4),
 (4, 4, 14),
 (14, 4, 16),
 (16, 4, 64),
 (64, 4, 16),
 (16, 4, 4),
 (4, 4, 1)]

In [197]:
import mps_nosym

qbonds,rmps_sites,rwfuns = mps_nosym.sweep_projection(sites,ne=mol.nelectron,tm=0,debug=False)

mps_nosym.dumpMPSforFOCUS(qbonds,rmps_sites,rwfuns,prefix='./rmps',debug=True)


[sweep_projection] nsite= 7 nroot= 1
 key= (10, 0) eig= [1.00000000e+00 1.12564737e-16 5.31271029e-25 4.84151827e-25] nkept= 1
rwfuns:
 [[1.]]

[dumpMPSforFOCUS] nsite= 7
 i= 0 (dl,dn,dr)= (4, 4, 1) nnz= 4
 i= 1 (dl,dn,dr)= (6, 4, 4) nnz= 13
 i= 2 (dl,dn,dr)= (9, 4, 6) nnz= 23
 i= 3 (dl,dn,dr)= (16, 4, 9) nnz= 72
 i= 4 (dl,dn,dr)= (14, 4, 16) nnz= 98
 i= 5 (dl,dn,dr)= (4, 4, 14) nnz= 34
 i= 6 (dl,dn,dr)= (1, 4, 4) nnz= 4
 rwfuns.shape= (1, 1)


0